In [ ]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai

See the getting started guide for more information:
https://ai.google.dev/gemini-api/docs/get-started/python
"""
import json 
import os
import re
import google.generativeai as genai

genai.configure(api_key="*************")  # AIzaSyAsjMpz***********awY7gyHGCRc5w

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = model.start_chat(
  history=[]
)

In [ ]:
input = """
Ai đã từng nghe những giai điệu tha thiết, ngọt ngào trong khúc ca “Nhật kí của mẹ” chắc không thể nào quên “Này con yêu ơi, con biết không? Mẹ yêu con, yêu con rất nhiều! Những kỷ niệm lần đầu yêu, suốt một đời đâu dễ quên…” Người mẹ nào cũng yêu con rất nhiều, yêu con vô bờ bến. Mỗi lần nghe những ca từ về mẹ, trong em lại hiện về hình ảnh người mẹ thân thương của mình.

Mẹ em năm nay vừa tròn bốn mươi, độ tuổi chưa phải quá già nhưng trên gương mặt đã hằn in bao vết tích mệt nhọc của thời gian. Mẹ có chiều cao khiêm tốn, nhưng vẻ nhỏ nhắn giúp thân hình mẹ cân đối. Mái tóc dài ngang vai, đen láy, suôn mượt càng tôn lên làn da trắng trẻo của mẹ. Làn da ấy đã có vài nếp nhăn cùng những chấm tàn nhang li ti trên gương mặt. Em hiểu những nếp nhăn đó là bao vất vả mẹ gánh chịu để nuôi dạy anh em em. Gương mặt mẹ trái xoan, xinh đẹp. Mẹ có đôi mắt nâu đen sâu thẳm. Người ta bảo, những người mắt sâu là những người sống tình cảm. Quả thực, mẹ em rất tình cảm, lúc nào trong mẹ cũng đầy ắp tình thương yêu cho gia đình, cho chúng em và cho những thế hệ học trò mà mẹ dạy dỗ. Đôi môi trái tim hồng hồng của mẹ lúc nào cũng nở nụ cười rạng rỡ. Chỉ khi anh em em chưa vâng lời, nụ cười đó mới vụt tắt. Khi đó, chiếc mũi cao của mẹ cũng đỏ hoe vì rơi nước mắt, trông mẹ buồn đến lạ. Em thích nhất chiếc má lúm đồng tiền của mẹ. Chiếc má lúm làm gương mặt mẹ thêm phần tươi xinh, duyên dáng.

Mẹ em thường mặc quần vải sẫm màu cùng áo sơ mi dài tay. Trong bộ trang phục này, trông mẹ rất nghiêm túc nhưng không kém phần trẻ trung. Những ngày lễ, mẹ thướt tha trong tà áo dài. Em yêu nhất là đôi bàn tay mẹ, đôi bàn tay mềm mại nhưng đã có chút nhăn nheo. Làm sao mà không nhăn nheo khi đôi bàn tay ấy đã chăm bẵm từng miếng ăn, giấc ngủ cho anh em em, đã dắt dìu bao thế hệ học trò bỡ ngỡ vào lớp một? Mẹ em là một người vừa ấm áp, vừa nghiêm khắc. Ngày còn nhỏ, mẹ mua cho chúng tôi rất nhiều đồ chơi. Nhưng mẹ cũng căn dặn sau khi chơi xong phải cất ngăn nắp và gìn giữ cẩn thận.

Bài hát “Nhật kí của mẹ” kể lại cả chặng đường người mẹ sinh thành, nuôi dưỡng và thương yêu người con nhưng có lẽ chẳng câu từ nào có thể đong đếm được sự hi sinh và tình cảm bao la mẹ dành cho con. Em luôn thầm hứa sẽ ngoan ngoãn, vâng lời cha mẹ và học tập thật tốt để không phụ lòng sinh dưỡng của mẹ.
"""



table_score =  """Mở bài- Giới thiệu về mẹ và cảm nhận chung - 1
Thân bài | Ngoại hình của mẹ |Tả chi tiết về khuôn mặt, mái tóc, đôi tay, dáng người | 2
Thân bài | Tính cách của mẹ | Miêu tả tính cách, sự chăm sóc, yêu thương và hy sinh của mẹ | 2
Kỷ niệm đáng nhớ | Kể lại kỷ niệm sâu sắc với mẹ và cảm xúc cá nhân. | 1.5
Kết bài | Bày tỏ tình cảm, lời hứa hoặc mong ước về mẹ. | 1
Yêu cầu thêm | Diễn đạt và sáng tạo | Câu văn rõ ràng, diễn đạt mạch lạc, sử dụng từ ngữ phong phú. | 0.25
Yêu cầu thêm | Chính tả và ngữ pháp | Viết đúng chính tả, đúng ngữ pháp. | 0.25"""

prompt = f"""
    Dựa vào than điểm đánh giá, hãy chấm bài văn ở dưới, và trả lại kết quả đúng format của "return", với những mục nào mà bạn cảm thấy chưa đạt yêu cầu, thì có thể chấm giảm đi 0.25 điểm
    
    **Than điểm đánh giá:**
    
        {table_score}
    
    **Bài văn:** 
        {input}
        
    **return**
        "Điểm tổng hợp": điểm,
        "Lí do trừ điểm": Lý do,
        "Nhận xét": nhận xét

"""

In [ ]:
response = chat_session.send_message(prompt)
context = response.text
data = context.split("\n")

In [81]:

json_data = []
for idx, value in enumerate(data):
    if data[idx - 1] == "```json":
        while True:
            if data[idx] == "```":
                break
            json_data.append(data[idx])
            idx += 1
            
json_string = "\n".join(json_data)

structured_data = json.loads(json_string)

total_score = sum([float(i) for i in structured_data['Điểm tổng hợp'].values()])

print("Điểm tổng hợp")
for key, value in structured_data['Điểm tổng hợp'].items():
    print(f"{key}: {value}")
print()
print(f"Tổng điểm: {total_score}")
print()

print('Lí do trừ điểm',"\n",structured_data['Lí do trừ điểm'])
print()
print('Nhận xét',"\n",structured_data['Nhận xét'])


Điểm tổng hợp
Mở bài: 1
Ngoại hình: 2
Tính cách: 2
Kỷ niệm đáng nhớ: 1
Kết bài: 1
Diễn đạt và sáng tạo: 0.25
Chính tả và ngữ pháp: 0.25

Tổng điểm: 7.5

Lí do trừ điểm 
 Phần 'Kỷ niệm đáng nhớ' chỉ khái quát về sự hi sinh của mẹ, chưa kể lại kỷ niệm cụ thể.

Nhận xét 
 Bài văn miêu tả ngoại hình, tính cách của mẹ khá chi tiết, sử dụng nhiều hình ảnh, chi tiết cụ thể, gợi tả và giàu cảm xúc. Tuy nhiên, phần 'Kỷ niệm đáng nhớ' chưa đạt yêu cầu, bài văn chưa có sự kết nối liền mạch giữa các phần. Cần trau chuốt thêm về mặt ngôn ngữ, sử dụng từ ngữ phong phú, diễn đạt tinh tế, gọn gàng hơn.
